<a href="https://colab.research.google.com/github/shaziyakhann/shaziyakhann/blob/main/qn3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
import requests

# Replace with your OpenRouteService API key
ORS_API_KEY = '5b3ce3597851110001cf6248ac9bd7689924491b812a74de277c35ca'

def get_geocode(location):
    geocode_url = f"https://api.openrouteservice.org/geocode/search?api_key={ORS_API_KEY}&text={location}"
    response = requests.get(geocode_url)

    if response.status_code != 200:
        raise Exception(f"Error fetching geocode: {response.status_code}")

    data = response.json()
    if 'features' not in data or len(data['features']) == 0:
        raise Exception("No geocoding results found.")

    coords = data['features'][0]['geometry']['coordinates']
    return coords

def get_traffic_data(start, end):
    try:
        start_coords = get_geocode(start)
        end_coords = get_geocode(end)
    except Exception as e:
        return {'error': str(e)}

    routing_url = f"https://api.openrouteservice.org/v2/directions/driving-car"
    headers = {
        'Authorization': ORS_API_KEY,
        'Content-Type': 'application/json'
    }
    body = {
        "coordinates": [start_coords, end_coords]
    }

    response = requests.post(routing_url, json=body, headers=headers)

    if response.status_code != 200:
        return {'error': f"Error fetching route: {response.status_code}"}

    route_data = response.json()

    if 'routes' not in route_data:
        return {'error': 'No routes found'}

    route = route_data['routes'][0]
    summary = route['summary']

    traffic_info = {
        'start_address': start,
        'end_address': end,
        'distance': summary['distance'] / 1000,  # convert to km
        'duration': summary['duration'] / 60,  # convert to minutes
        'polyline': route['geometry']
    }

    return traffic_info

def display_traffic_info(start, end):
    traffic_info = get_traffic_data(start, end)
    if 'error' in traffic_info:
        print("Error:", traffic_info['error'])
    else:
        print("Traffic Information:")
        print(f"Start Address: {traffic_info['start_address']}")
        print(f"End Address: {traffic_info['end_address']}")
        print(f"Distance: {traffic_info['distance']} km")
        print(f"Duration: {traffic_info['duration']} minutes")
        print("Polyline:", traffic_info['polyline'])

# Hardcoded start and end locations
start = "New York, NY"
end = "Los Angeles, CA"
display_traffic_info(start, end)

Traffic Information:
Start Address: New York, NY
End Address: Los Angeles, CA
Distance: 4500.8268 km
Duration: 2694.7033333333334 minutes
Polyline: m{hwFtlnbME?eALOBuD`@kCZsApE_A~CWx@uAvEsApEy@pC_@lAeAlDUr@m@vBkAzDoAhEGPc@~AYNyDtAwGjCoCfAqHtCUHu@ZmDrASHsAj@e@PsAj@_@PyEjBsAd@WP}@b@e@VkHfDkzA~q@qDbBMFi@VwA~@]d@[z@u@xCA^w@zCy@bDOf@KTy@pA}A~BqArBiBrCwAxBgBnCeBjCW`@_AzAcBhC}@xAg@v@g@Fg@IoCw@}Bu@WjFGZS|@Wl@QX]ZQJ_@HeBKU@]Nm@\WXOTM\K^M`AqC`m@aEdb@gDp_@}Ap`@IjAaBlLUpAYz@o@vA}BbEy@fAc@x@_AdCGZCt@QlEGdAEx@Cj@Cd@G~AATATEr@Ct@E`AMrCE|@A^ANG~@SzEG~BDhBFz@bAbNp@xFb@xCFnAAp@Ex@Gl@Ov@c@pAy@zAk@x@wc@pi@s@dAy@dB[t@q@tBe@tBWvAQdB]bDKvBGv@Qj@k@O[ScBaBkA_Ae@YeAg@oA[sH_BYGkAUg@KaASw@Om@MuA[k@OqCu@qCs@aAa@}BaAwCoAoDgB{@o@}@w@}A_ByCkDa@a@qDoCkLaIaAs@UMgCgBc@YiAw@kBuA}@{@aKuK_ByAsFwD_IaFaBcAe@YeAq@i@]uBsAyA{@{BuAcDsBg@]WSgBkAqEaDy@m@eEuCgBmA_BkAaB}@aAm@o@i@q@e@cAk@cAk@{@_@_AQm@A_AJSFm@b@oAdBORsAfBcAjAeBdB}AhAoAr@eAh@gBl@oAZkRhEoANoQxDkCn@sA^kCbAwBdA{@f@qBrA_Ar@cBxAu@t@cBpB_BvBkPjWaJrNaBbCsJ~N_D`F_JzMi[xe@oAhB_